In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# IMport Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.api import adfuller
from statsmodels.tsa.api import rmse, aic

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
filepath = "https://raw.githubusercontent.com/selva86/datasets/master/Raotbl6.csv"

df = pd.read_csv(filepath, parse_dates=['dates'], index_col='date')
print(df.shape) # (123,8)
df.tail()

In [ ]:
# Plot
fig, axes = plt.subplots(nrows=4, ncols=2, dpi=120, figsize=(10,6))
for i, ax in enumerate(axes.flatten()):
    data = df[df.columns[i]]
    ax.plot(data, color='red', linewidth=1)
    #Decorations
    ax.set_title(df.columns[i])
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

plt.tight_layout()